# Trabalho 1 : Tetraedro gerado aleatoriamente numa esfera
## SME0123 - Estatística
### Outubro de 2023
* Artur De Vlieger Lima - 13671574
* Gabriel da Costa Merlin - 12544420
* Vicenzo D'Arezzo Zilio - 13671790

### Descrição do problema:
Queremos ser capazes de estudar a probabilidade do seguinte evento: escolhendo 4 pontos aleatoriamente na superfície de uma esfera, qual a probabilidade do centro dela estar contido no tetraedro formado ao tomar esses pontos como vértices.

## 1. Definições, funções e classes

In [1]:
# Baixando pacote que permite fazer animações.
!pip install -q ipympl

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

%matplotlib widget

ModuleNotFoundError: No module named 'google'

In [ ]:
# Constantes usadas durante o código.
n_points = 4
radius = 1

In [ ]:
# Classe que representa os pontos que serão gerados ao redor da esfera.
class SpherePoint():

  # Construtor da classe. Usa modality para saber de que modo gerará o ponto.
  def __init__(self, modality = 1):
    # Gerando usando coordenadas esféricas.
    if modality == 1:
      self.theta = (np.pi * 2 * np.random.rand(1))[0]
      self.phi = (np.pi * np.random.rand(1))[0]
      self.rho = radius
    # Gerando usando pontos em um cubo.
    else:
      norm = 2
      while norm > 1:
        self.x = ((np.random.rand(1))[0] * 2) - 1
        self.y = ((np.random.rand(1))[0] * 2) - 1
        self.z = ((np.random.rand(1))[0] * 2) - 1
        arr = [self.x, self.y, self.z]
        norm = np.linalg.norm(arr)

      self.x = self.x / norm
      self.y = self.y / norm
      self.z = self.z / norm
    self.modality = modality

  # Retorna as coordenadas cartesianas do ponto da esfera em uma lista. Se for
  # gerado via coordenadas polares, converte-as, se não, só as retorna.
  def toCartesianCoord(self):
    if self.modality == 1:
      self.x = np.cos(self.theta) * np.sin(self.phi) * self.rho
      self.y = np.sin(self.theta) * np.sin(self.phi) * self.rho
      self.z = np.cos(self.phi) * self.rho

    return [self.x, self.y, self.z]

# Classe que representa o Tetraedro.
class Tetrahedron():

  # Construtor da classe.
  def __init__(self, modality = 1):
    isTetrahedron = False
    self.points = []

    # Verificamos se o Tetraedro gerado é realmente um tetraedro ao ver se os
    # seus pontos são coplanares ou não. Caso não seja válido, geramos outros pontos.
    while(not isTetrahedron):
      self.points.clear()

      for i in range(4):
        sp = SpherePoint(modality)
        self.points.append(sp)

      if(areLinIndep(self.points[0].toCartesianCoord(), self.points[1].toCartesianCoord(), self.points[2].toCartesianCoord(), self.points[3].toCartesianCoord())):
        isTetrahedron = True

  # Retorna os vértices do tetraedro.
  def getPoints(self):
    return list(map(SpherePoint.toCartesianCoord, self.points))

In [ ]:
# Retorna as coordenadas cartesinas de um ponto dado em coordenadas esféricas.
def esferical_to_cartesian(th, ph, rad):
  x = np.cos(th) * np.sin(ph) * rad
  y = np.sin(th) * np.sin(ph) * rad
  z = np.cos(ph) * rad
  return (x, y, z)

# Verifica se 4 pontos são linearmente dependentes ou não.
def areLinIndep(pa, pb, pc, pd):
  v1 = np.array(pa)
  v2 = np.array(pb)
  v3 = np.array(pc)
  v4 = np.array(pd)

  dim = np.linalg.matrix_rank(np.column_stack((v1, v2, v3, v4)))

  if dim == 3 : return True;
  else : return False;

# Plota o tetraedro gerado.
def plotTetrahedron(points):
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # Gerando a esfera.
  u = np.linspace(0, 2 * np.pi, 100)
  v = np.linspace(0, np.pi, 100)
  x = radius * np.outer(np.cos(u), np.sin(v))
  y = radius * np.outer(np.sin(u), np.sin(v))
  z = radius * np.outer(np.ones(np.size(u)), np.cos(v))

  # Plotando a superfície.
  ax.plot_surface(x, y, z, color='linen', alpha=0.2)

  # Plotando curvas circulares na superfíce.
  theta = np.linspace(0, 2 * np.pi, 100)
  z = np.zeros(100)
  x = radius * np.sin(theta)
  y = radius * np.cos(theta)

  ax.plot(x, y, z, color='black', alpha=0.15)
  ax.plot(z, x, y, color='black', alpha=0.15)

  # Adicionando as linhas dos eixos coordenados.
  zeros = np.zeros(1000)
  line = np.linspace(-10,10,1000)

  # Adicionando os pontos.
  x1 = [points[0][0], points[1][0], points[2][0]]
  y1 = [points[0][1], points[1][1], points[2][1]]
  z1 = [points[0][2], points[1][2], points[2][2]]

  x2 = [points[0][0], points[1][0], points[3][0]]
  y2 = [points[0][1], points[1][1], points[3][1]]
  z2 = [points[0][2], points[1][2], points[3][2]]

  x3 = [points[0][0], points[2][0], points[3][0]]
  y3 = [points[0][1], points[2][1], points[3][1]]
  z3 = [points[0][2], points[2][2], points[3][2]]

  x4 = [points[1][0], points[2][0], points[3][0]]
  y4 = [points[1][1], points[2][1], points[3][1]]
  z4 = [points[1][2], points[2][2], points[3][2]]

  for p in points:
    ax.scatter(p[0], p[1], p[2], color='r', marker='o', s=40)
    ax.scatter(0, 0, 0, color='black', marker='o', s=40)

  # Adicionando o polígono.
  verts1 = [list(zip(x1,y1,z1))]
  verts2 = [list(zip(x2,y2,z2))]
  verts3 = [list(zip(x3,y3,z3))]
  verts4 = [list(zip(x4,y4,z4))]

  ax.add_collection3d(Poly3DCollection(verts1, facecolors='cyan', linewidths=1, edgecolors='r', alpha=.25))
  ax.add_collection3d(Poly3DCollection(verts2, facecolors='yellow', linewidths=1, edgecolors='r', alpha=.25))
  ax.add_collection3d(Poly3DCollection(verts3, facecolors='red', linewidths=1, edgecolors='r', alpha=.25))
  ax.add_collection3d(Poly3DCollection(verts4, facecolors='green', linewidths=1, edgecolors='r', alpha=.25))

  ax.plot(line, zeros, zeros, color='black', alpha=0.25)
  ax.plot(zeros, line, zeros, color='black', alpha=0.25)
  ax.plot(zeros, zeros, line, color='black', alpha=0.25)

  plt.show()

## 2. Gerando os pontos aleatoriamente na superfície:


 Modelo 1

Para gerar os pontos, representamo-nos no sistema de coordenadas esféricas, gerando os ângulos Phi e Theta aleatorimente e, para garantir o pertencimento à superfície, usamos um raio fixo equivalente ao raio da esfera.

* ϴ ∈ [0, 2π]
* ϕ ∈ [0, π]

Posteriormente, usaremos uma transformação para converter o sistema de coordenadas para as cartesianas no Espaço.

In [ ]:
# Gerando vários pontos ao redor da esfera.
theta_list = np.pi * 2 * np.random.rand(n_points)
phi_list = np.pi * np.random.rand(n_points)

points = []

# Mudando o sistema de coordenadas de eséricas para cartesianas.
for th, ph in zip(theta_list, phi_list):
  points.append(esferical_to_cartesian(th, ph, radius))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Gerando a esfera.
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))

# Plotando a superfície.
ax.plot_surface(x, y, z, color='linen', alpha=0.5)

# Plotando curvas circulares na superfíce.
theta = np.linspace(0, 2 * np.pi, 100)
z = np.zeros(100)
x = radius * np.sin(theta)
y = radius * np.cos(theta)

ax.plot(x, y, z, color='black', alpha=0.75)
ax.plot(z, x, y, color='black', alpha=0.75)

# Adicionando as linhas dos eixos coordenados.
zeros = np.zeros(1000)
line = np.linspace(-10,10,1000)

# Adicionando os pontos.
for p in points:
  ax.scatter(p[0], p[1], p[2], color='r', marker='o', s=40)

ax.plot(line, zeros, zeros, color='black', alpha=0.75)
ax.plot(zeros, line, zeros, color='black', alpha=0.75)
ax.plot(zeros, zeros, line, color='black', alpha=0.75)

plt.show()

Modelo 2

Neste segundo modelo, vamos gerar 𝑥 ∼ 𝑈 (−1, 1) e assim, analogamente, para 𝑦, 𝑧, criando um ponto
𝑝1 uniformemente distribuído dentro de um cubo. Descartamos 𝑝1 se ele cair fora da esfera de
raio 𝑟 = 1 inscrito no cubo. Por ultimo, “projetamos” na superfície: 𝑝1 = 𝑝1/ norma(𝑝1). E, claro,
repete-se isso para obter 𝑝2 a 𝑝4, levando ao primeiro tetraedro simulado por este modelo.


In [ ]:
# Gerando vários pontos ao redor da esfera.
points = []
for i in range(4):
    sp = SpherePoint(modality = 2)
    points.append(sp.toCartesianCoord())

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Gerando a esfera.
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))

# Plotando a superfície.
ax.plot_surface(x, y, z, color='linen', alpha=0.5)

# Plotando curvas circulares na superfíce.
theta = np.linspace(0, 2 * np.pi, 100)
z = np.zeros(100)
x = radius * np.sin(theta)
y = radius * np.cos(theta)

ax.plot(x, y, z, color='black', alpha=0.75)
ax.plot(z, x, y, color='black', alpha=0.75)

# Adicionando as linhas dos eixos coordenados.
zeros = np.zeros(1000)
line = np.linspace(-10,10,1000)

# Adicionando os pontos.
for p in points:
  ax.scatter(p[0], p[1], p[2], color='g', marker='o', s=40)

ax.plot(line, zeros, zeros, color='black', alpha=0.75)
ax.plot(zeros, line, zeros, color='black', alpha=0.75)
ax.plot(zeros, zeros, line, color='black', alpha=0.75)

plt.show()

## 3. Obtenção do Tetraedro
Para aferir o sólido formado, primeiro, devemos verificar a dependência linear dos vetores dados entre cada ponto e a origem. Geometricamente, o tetraedro só se forma caso os 4 pontos não pertençam ao mesmo plano, o que pode ser verificado através da medição da dimensão.

In [ ]:
# Gerando o Tetraedro.
tet = Tetrahedron()

# Encontrando as coordenadas cartesianas de cada um dos 4 vértices do tetraedro.
vertexA = np.array(tet.points[0].toCartesianCoord())
vertexB = np.array(tet.points[1].toCartesianCoord())
vertexC = np.array(tet.points[2].toCartesianCoord())
vertexD = np.array(tet.points[3].toCartesianCoord())

# Calculando os vetores normais de cada face do Tetraedro.
normalVectors = [
  np.cross((vertexB - vertexA), (vertexC - vertexA)),
  np.cross((vertexB - vertexA), (vertexD - vertexA)),
  np.cross((vertexC - vertexA), (vertexD - vertexA)),
  np.cross((vertexC - vertexB), (vertexD - vertexB))
]

# Encontrando o centróide do Tetraedro.
centroid = (vertexA + vertexB + vertexC + vertexD) / 4

# Lista com todos os vértices do Tetraedro.
basePoints = np.array([vertexA, vertexA, vertexA, vertexB])

# Fazendo com que todos os vetores normais apontem para fora do tetraedro.
for i in range(4):
  if(((centroid - basePoints[i]) * normalVectors[i]).sum() > 0):
    normalVectors[i] *= -1

isOutside = False

# Verificando se o centro da esfera está dentro do Tetraedro.
for i in range(4):
  if(((-1 * basePoints[i]) * normalVectors[i]).sum() > 0):
    isOutside = True
    break

# Plotando o Tetraedro.
plotTetrahedron(tet.getPoints())

In [ ]:
# Imprimindo se o centro da esfera está fora do Tetraedro.
isOutside

## 4. Simulação

In [ ]:
# Descomente qual a modalidade com a qual você deseja gerar os pontos do Tetraedro.
#   1 - Coordenadas Esféricas
#   2 - Auxílio de um cubo

# modality = 1
modality = 2

In [ ]:
# Variáveis contadoras.
totalinside = 0
next = 100

# Simulando o teste 10M de vezes.
for i in range(10000000):

  tet = Tetrahedron(modality = modality)

  vertexA = np.array(tet.points[0].toCartesianCoord())
  vertexB = np.array(tet.points[1].toCartesianCoord())
  vertexC = np.array(tet.points[2].toCartesianCoord())
  vertexD = np.array(tet.points[3].toCartesianCoord())

  normalVectors = [
    np.cross((vertexB - vertexA), (vertexC - vertexA)),
    np.cross((vertexB - vertexA), (vertexD - vertexA)),
    np.cross((vertexC - vertexA), (vertexD - vertexA)),
    np.cross((vertexC - vertexB), (vertexD - vertexB))
  ]

  centroid = (vertexA + vertexB + vertexC + vertexD) / 4

  basePoints = np.array([vertexA, vertexA, vertexA, vertexB])

  for j in range(4):
    if(((centroid - basePoints[j]) * normalVectors[j]).sum() > 0):
      normalVectors[j] *= -1

  isOutside = False
  for j in range(4):
    if(((-1 * basePoints[j]) * normalVectors[j]).sum() > 0):
      isOutside = True
      break
  if isOutside == False:
    totalinside += 1

  if i == (next - 1):
    print("Percentage of " + str(next) + " samples")
    print(totalinside / next)
    next *= 10